In [1]:
data_var = '2023-12-23'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10258,2023-12-23,Brasil Nbb,11:00,Brasília,São Paulo,3.69,1.25,160.5,1.86,1.84,7.5,2.06,1.68,rmKzyCR1,0.271003,0.800000,0.537634,0.543478,0.071003,0.6,0.6,NaN,NaN,464.468,253.893632,0.546633,114.448,40.466944,0.353584,175.266,121.158,249.09,99.90,0.0,0.0,0.0,0.0,0.698518,0.007644,0.143690,-2.81,-0.562,-4.786477,0.0,0.0,0.0,-0.75,-0.150,-1.666667,0.0,0.0,0.0
10259,2023-12-23,Brasil Nbb,11:00,R10 Score Vasco da Gama,Fortaleza B.C.,1.44,2.65,147.5,1.88,1.79,-6.5,2.09,1.66,4dLWyhCe,0.694444,0.377358,0.531915,0.558659,0.071803,0.2,0.6,NaN,NaN,176.784,44.723626,0.252985,200.908,134.962348,0.671762,132.928,266.996,153.90,352.24,0.0,0.0,0.0,0.0,0.418386,0.034681,0.162163,1.11,0.222,1.981982,0.0,0.0,0.0,0.97,0.194,8.505155,0.0,0.0,0.0
10260,2023-12-23,Brasil Nbb,17:00,Flamengo,Unifacisa,1.19,4.37,155.5,1.84,1.86,-11.5,1.97,1.75,dj8Iae42,0.840336,0.228833,0.543478,0.537634,0.069169,0.4,0.6,NaN,NaN,119.158,26.945149,0.226130,117.088,15.705379,0.134133,109.238,157.226,89.32,137.28,0.0,0.0,0.0,0.0,0.808849,0.007644,0.083636,1.18,0.236,0.805085,0.0,0.0,0.0,-2.18,-0.436,-7.729358,0.0,0.0,0.0
10261,2023-12-23,Brasil Nbb,17:00,São José,Franca,2.56,1.47,161.5,1.85,1.85,3.5,2.10,1.66,AgCMbFJ8,0.390625,0.680272,0.540541,0.540541,0.070897,0.8,0.2,NaN,NaN,112.094,11.520659,0.102777,100.640,20.611448,0.204804,156.182,116.130,107.30,82.50,0.0,0.0,0.0,0.0,0.382504,0.000000,0.165493,2.96,0.592,2.635135,0.0,0.0,0.0,-0.46,-0.092,-5.108696,0.0,0.0,0.0
10262,2023-12-23,Brasil Nbb,17:00,União Corinthians,Pinheiros,2.12,1.67,154.5,1.81,1.86,1.5,1.96,1.75,hI1RcZYE,0.471698,0.598802,0.552486,0.537634,0.070501,0.2,0.8,NaN,NaN,247.710,72.655561,0.293309,144.964,32.290563,0.222749,173.018,234.288,207.46,106.88,0.0,0.0,0.0,0.0,0.167915,0.019267,0.080050,-3.63,-0.726,-1.542700,0.0,0.0,0.0,-0.13,-0.026,-25.769231,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10497,2023-12-23,Itália Série A1 Feminina,15:00,Ragusa F,Battipaglia F,1.05,8.80,139.5,1.88,1.79,-26.5,2.08,1.66,Yc8D4Bf4,0.952381,0.113636,0.531915,0.558659,0.066017,0.0,0.0,NaN,NaN,207.700,175.508947,0.845012,NaN,NaN,NaN,0.000,0.000,105.28,654.50,0.0,1.0,0.0,0.0,1.112706,0.034681,0.158815,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
10498,2023-12-23,Lituânia Nkl,11:00,Suduva-Mantinga,Rytas 2,1.02,11.60,166.5,1.84,1.84,-27.5,2.08,1.66,vN7HxMoF,0.980392,0.086207,0.543478,0.543478,0.066599,0.6,0.8,NaN,NaN,125.828,20.339800,0.161648,356.468,209.416093,0.587475,128.504,383.530,103.88,652.47,0.0,1.0,0.0,0.0,1.185609,0.000000,0.158815,2.46,0.492,0.040650,0.0,0.0,0.0,5.25,1.050,10.095238,0.0,0.0,0.0
10499,2023-12-23,Portugal Lpb,18:00,Oliveirense,Vitória SC,1.12,5.61,156.5,1.86,1.84,-13.5,2.03,1.70,YDIn7ZXl,0.892857,0.178253,0.537634,0.543478,0.071110,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,128.96,138.38,0.0,0.0,0.0,0.0,0.943509,0.007644,0.125118,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
10500,2023-12-23,Romênia Divisão A,13:00,Cluj-Napoca,Dinamo Bucharest,1.06,7.70,163.5,1.86,1.84,-21.5,2.07,1.67,63bDZEL6,0.943396,0.129870,0.537634,0.543478,0.073266,0.0,0.0,NaN,NaN,NaN,NaN,NaN,137.974,45.687827,0.331134,0.000,0.000,88.88,112.34,0.0,0.0,0.0,0.0,1.071961,0.007644,0.151253,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
8,00:00,Eua Nba,Golden State Warriors,Washington Wizards,246.5,1.89,1.0000,Over
9,00:00,Eua Nba,Sacramento Kings,Phoenix Suns,243.5,1.89,1.0000,Over
10,14:30,Eua Nba,New York Knicks,Milwaukee Bucks,238.5,1.81,0.9999,Over
12,21:00,Eua Nba,Charlotte Hornets,Denver Nuggets,225.5,1.86,1.0000,Over
13,21:00,Eua Nba,Indiana Pacers,Orlando Magic,241.5,1.84,1.0000,Over
14,21:30,Eua Nba,Atlanta Hawks,Memphis Grizzlies,238.5,1.81,1.0000,Over
15,21:30,Eua Nba,Brooklyn Nets,Detroit Pistons,231.5,1.83,0.9999,Over
16,21:30,Eua Nba,Toronto Raptors,Utah Jazz,228.5,1.88,1.0000,Over
237,21:00,Eua Nba,New Orleans Pelicans,Houston Rockets,222.5,1.86,1.0000,Over
238,22:30,Eua Nba,Dallas Mavericks,San Antonio Spurs,238.5,1.89,1.0000,Over
